# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
from scipy import stats
import math

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [3]:
stocks = pd.read_csv("sp_500_stocks.csv")
from secret import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [9]:
symbol = "AAPL"
api_url = f"https://cloud.iexapis.com/v1/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}"
response = requests.get(api_url)
data = response.json()
print(data)

{'avgTotalVolume': 64672410, 'calculationPrice': 'tops', 'change': -3.91, 'changePercent': -0.02375, 'close': None, 'closeSource': 'official', 'closeTime': None, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': None, 'highSource': None, 'highTime': None, 'iexAskPrice': 160.76, 'iexAskSize': 100, 'iexBidPrice': 160.74, 'iexBidSize': 100, 'iexClose': 160.75, 'iexCloseTime': 1681137897292, 'iexLastUpdated': 1681137897292, 'iexMarketPercent': 0.015369934879462293, 'iexOpen': 161.415, 'iexOpenTime': 1681133400012, 'iexRealtimePrice': 160.75, 'iexRealtimeSize': 200, 'iexVolume': 258648, 'lastTradeTime': 1681137896743, 'latestPrice': 160.75, 'latestSource': 'IEX real time price', 'latestTime': '10:44:57 AM', 'latestUpdate': 1681137897292, 'latestVolume': None, 'low': None, 'lowSource': None, 'lowTime': None, 'marketCap': 2543377819500, 

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [11]:
price = data['latestPrice']
pe_ratio = data['peRatio']
pe_ratio

27.34

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [13]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD
GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILM

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [28]:
import warnings
warnings.filterwarnings("ignore")
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/v1/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        if symbol == 'HFC' or symbol == 'WLTW' or symbol == 'DISCA':
            continue
        else:
            final_dataframe =  final_dataframe.append(
                                        pd.Series([
                                                   
                                                   symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['quote']['peRatio'],
                                                   'N/A'
                                            
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        

In [29]:
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,135.980,30.7,N/A
1,AAL,14.025,72.33,N/A
2,AAP,120.290,14.55,N/A
3,AAPL,160.840,27.35,N/A
4,ABBV,160.440,24.2,N/A
...,...,...,...,...
497,YUM,131.120,28.69,N/A
498,ZBH,127.040,115.49,N/A
499,ZBRA,297.195,33.73,N/A
500,ZION,29.330,3.03,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [30]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace = True)

In [35]:
final_dataframe.head(5)

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,FRC,13.890,1.69,N/A
1,DISH,8.685,2.41,N/A
2,ZION,29.330,3.03,N/A
3,APA,38.800,3.52,N/A
4,AIG,50.465,3.88,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [32]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [33]:
portfolio_input()

Enter the value of your portfolio: 200000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [34]:
portfolio_size

'200000'

## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [44]:
symbol = 'AAPL'
batch_api_call_url = f'https://cloud.iexapis.com/v1/stock/market/batch/?types=stats,quote,advanced-stats&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# P/E Ratio
pe_ratio = data[symbol]['quote']['peRatio']

# P/B Ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#P/S Ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# EV/EBITDA
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda
ev_to_ebitda

# EV/GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

15.970400059686824

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [51]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/v1/stock/market/batch/?types=stats,quote,advanced-stats&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        if symbol == 'HFC' or symbol == 'WLTW' or symbol == 'DISCA':
            continue
        else:
            enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
            ebitda = data[symbol]['advanced-stats']['EBITDA']
            gross_profit = data[symbol]['advanced-stats']['grossProfit']

            try:
                ev_to_ebitda = enterprise_value/ebitda
            except TypeError:
                ev_to_ebitda = np.NaN

            try:
                ev_to_gross_profit = enterprise_value/gross_profit
            except TypeError:
                ev_to_gross_profit = np.NaN

            rv_dataframe = rv_dataframe.append(
                pd.Series([
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    'N/A',
                    data[symbol]['quote']['peRatio'],
                    'N/A',
                    data[symbol]['advanced-stats']['priceToBook'],
                    'N/A',
                    data[symbol]['advanced-stats']['priceToSales'],
                    'N/A',
                    ev_to_ebitda,
                    'N/A',
                    ev_to_gross_profit,
                    'N/A',
                    'N/A'
            ],
            index = rv_columns),
                ignore_index = True
            )

In [52]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,137.255,N/A,30.98,N/A,7.28,N/A,5.89,N/A,21.592247,N/A,11.253102,N/A,N/A
1,AAL,14.130,N/A,72.87,N/A,-1.58,N/A,0.1868,N/A,9.490061,N/A,0.731943,N/A,N/A
2,AAP,120.140,N/A,14.53,N/A,2.67,N/A,0.6413,N/A,8.274709,N/A,1.664165,N/A,N/A
3,AAPL,161.625,N/A,27.49,N/A,45.93,N/A,6.72,N/A,21.270965,N/A,15.970400,N/A,N/A
4,ABBV,160.540,N/A,24.21,N/A,16.52,N/A,4.91,N/A,12.093854,N/A,8.342498,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497,YUM,130.875,N/A,28.64,N/A,-4.17,N/A,5.41,N/A,20.962494,N/A,9.512703,N/A,N/A
498,ZBH,127.690,N/A,116.08,N/A,2.25,N/A,3.89,N/A,18.437143,N/A,6.567511,N/A,N/A
499,ZBRA,301.185,N/A,34.19,N/A,5.65,N/A,2.67,N/A,16.185789,N/A,6.612487,N/A,N/A
500,ZION,29.780,N/A,3.07,N/A,0.9775,N/A,1.1,N/A,2.359150,N/A,1.137051,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [56]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
6,ABMD,381.020,N/A,65.69,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
28,ALXN,182.500,N/A,59.45,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,319.365,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,311.240,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
88,CERN,94.920,N/A,47.94,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.000,N/A,9.73,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
121,CTXS,103.900,N/A,39.96,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
135,DISCK,24.420,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
164,ETFC,49.260,N/A,14.27,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
185,FLIR,57.340,N/A,32.03,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [57]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [58]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [61]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

# Print each percentile score to make sure it was calculated properly
for metric in metrics.values():
    pass#print(rv_dataframe[metric])

#Print the entire DataFrame    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,137.255,N/A,30.98,0.7251,7.2800,0.766932,5.8900,0.846614,21.592247,0.854582,11.253102,0.836653,N/A
1,AAL,14.130,N/A,72.87,0.964143,-1.5800,0.061753,0.1868,0.011952,9.490061,0.280876,0.731943,0.025896,N/A
2,AAP,120.140,N/A,14.53,0.314741,2.6700,0.440239,0.6413,0.09761,8.274709,0.221116,1.664165,0.113546,N/A
3,AAPL,161.625,N/A,27.49,0.667331,45.9300,0.978088,6.7200,0.866534,21.270965,0.844622,15.970400,0.936255,N/A
4,ABBV,160.540,N/A,24.21,0.60259,16.5200,0.930279,4.9100,0.784861,12.093854,0.406375,8.342498,0.679283,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497,YUM,130.875,N/A,28.64,0.693227,-4.1700,0.057769,5.4100,0.818725,20.962494,0.828685,9.512703,0.747012,N/A
498,ZBH,127.690,N/A,116.08,0.978088,2.2500,0.373506,3.8900,0.709163,18.437143,0.752988,6.567511,0.482072,N/A
499,ZBRA,301.185,N/A,34.19,0.790837,5.6500,0.691235,2.6700,0.528884,16.185789,0.665339,6.612487,0.488048,N/A
500,ZION,29.780,N/A,3.07,0.089641,0.9775,0.111554,1.1000,0.193227,2.359150,0.02988,1.137051,0.063745,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [62]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,137.255,N/A,30.98,0.7251,7.2800,0.766932,5.8900,0.846614,21.592247,0.854582,11.253102,0.836653,0.805976
1,AAL,14.130,N/A,72.87,0.964143,-1.5800,0.061753,0.1868,0.011952,9.490061,0.280876,0.731943,0.025896,0.268924
2,AAP,120.140,N/A,14.53,0.314741,2.6700,0.440239,0.6413,0.09761,8.274709,0.221116,1.664165,0.113546,0.23745
3,AAPL,161.625,N/A,27.49,0.667331,45.9300,0.978088,6.7200,0.866534,21.270965,0.844622,15.970400,0.936255,0.858566
4,ABBV,160.540,N/A,24.21,0.60259,16.5200,0.930279,4.9100,0.784861,12.093854,0.406375,8.342498,0.679283,0.680677
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497,YUM,130.875,N/A,28.64,0.693227,-4.1700,0.057769,5.4100,0.818725,20.962494,0.828685,9.512703,0.747012,0.629084
498,ZBH,127.690,N/A,116.08,0.978088,2.2500,0.373506,3.8900,0.709163,18.437143,0.752988,6.567511,0.482072,0.659163
499,ZBRA,301.185,N/A,34.19,0.790837,5.6500,0.691235,2.6700,0.528884,16.185789,0.665339,6.612487,0.488048,0.632869
500,ZION,29.780,N/A,3.07,0.089641,0.9775,0.111554,1.1000,0.193227,2.359150,0.02988,1.137051,0.063745,0.09761


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [63]:
rv_dataframe.sort_values(by = 'RV Score', inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)

In [64]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,LNC,20.865,N/A,-1.59,0.079681,0.9687,0.109562,0.1884,0.013944,-1.742331,0.01992,0.188411,0.001992,0.04502
1,FRC,14.060,N/A,1.71,0.085657,0.1825,0.065737,0.3732,0.053785,1.080345,0.021912,0.379242,0.003984,0.046215
2,KSS,23.525,N/A,-148.61,0.011952,0.6669,0.079681,0.1387,0.003984,6.862881,0.165339,1.089215,0.059761,0.064143
3,GM,35.575,N/A,5.80,0.12749,0.7116,0.085657,0.3078,0.033865,1.549978,0.023904,1.165483,0.067729,0.067729
4,F,12.665,N/A,-25.66,0.033865,1.1200,0.13745,0.3065,0.031873,3.287782,0.043825,1.529666,0.101594,0.069721
5,AIG,50.995,N/A,3.92,0.093625,0.9483,0.10757,0.6640,0.103586,1.938334,0.025896,0.664028,0.023904,0.070916
6,UNM,39.160,N/A,6.02,0.135458,0.8332,0.091633,0.6391,0.095618,3.448032,0.049801,0.639055,0.01992,0.078486
7,WBA,36.025,N/A,-9.90,0.055777,1.4600,0.198207,0.2316,0.017928,-8.351098,0.017928,1.558830,0.103586,0.078685
8,SYF,28.840,N/A,4.69,0.109562,1.0100,0.119522,0.6528,0.101594,2.798942,0.033865,0.795831,0.035857,0.08008
9,PRU,84.100,N/A,-21.40,0.039841,1.8800,0.300797,0.5094,0.071713,-38.001198,0.007968,0.509425,0.00996,0.086056


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [65]:
portfolio_input()

Enter the value of your portfolio: 1000000


In [72]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
for row in range(0, len(rv_dataframe['Ticker'])):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][i])
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,LNC,20.865,958,-1.59,0.079681,0.9687,0.109562,0.1884,0.013944,-1.742331,0.01992,0.188411,0.001992,0.04502
1,FRC,14.060,1422,1.71,0.085657,0.1825,0.065737,0.3732,0.053785,1.080345,0.021912,0.379242,0.003984,0.046215
2,KSS,23.525,850,-148.61,0.011952,0.6669,0.079681,0.1387,0.003984,6.862881,0.165339,1.089215,0.059761,0.064143
3,GM,35.575,562,5.80,0.12749,0.7116,0.085657,0.3078,0.033865,1.549978,0.023904,1.165483,0.067729,0.067729
4,F,12.665,1579,-25.66,0.033865,1.1200,0.13745,0.3065,0.031873,3.287782,0.043825,1.529666,0.101594,0.069721
5,AIG,50.995,392,3.92,0.093625,0.9483,0.10757,0.6640,0.103586,1.938334,0.025896,0.664028,0.023904,0.070916
6,UNM,39.160,510,6.02,0.135458,0.8332,0.091633,0.6391,0.095618,3.448032,0.049801,0.639055,0.01992,0.078486
7,WBA,36.025,555,-9.90,0.055777,1.4600,0.198207,0.2316,0.017928,-8.351098,0.017928,1.558830,0.103586,0.078685
8,SYF,28.840,693,4.69,0.109562,1.0100,0.119522,0.6528,0.101594,2.798942,0.033865,0.795831,0.035857,0.08008
9,PRU,84.100,237,-21.40,0.039841,1.8800,0.300797,0.5094,0.071713,-38.001198,0.007968,0.509425,0.00996,0.086056


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [73]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [74]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [75]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [76]:
writer.save()